In [1]:
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



In [2]:
ASSAY=Tewhey_K562_TileMPRA
PREFIX=OL
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count

echo ${FD_INP}
ls ${FD_INP}/${PREFIX}*hg38*raw*rep*bed | xargs -n 1 basename

/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/Tewhey_K562_TileMPRA/fragment_count
OL13_20220512.hg38.raw.Input.rep1.stranded_pos.bed
OL13_20220512.hg38.raw.Input.rep2.stranded_pos.bed
OL13_20220512.hg38.raw.Input.rep3.stranded_pos.bed
OL13_20220512.hg38.raw.Input.rep4.stranded_pos.bed
OL13_20220512.hg38.raw.Output.rep1.stranded_pos.bed
OL13_20220512.hg38.raw.Output.rep2.stranded_pos.bed
OL13_20220512.hg38.raw.Output.rep3.stranded_pos.bed
OL13_20220512.hg38.raw.Output.rep4.stranded_pos.bed
OL43_20221003.hg38.raw.Input.rep1.stranded_pos.bed
OL43_20221003.hg38.raw.Input.rep2.stranded_pos.bed
OL43_20221003.hg38.raw.Input.rep3.stranded_pos.bed
OL43_20221003.hg38.raw.Input.rep4.stranded_pos.bed
OL43_20221003.hg38.raw.Input.rep5.stranded_pos.bed
OL43_20221003.hg38.raw.Input.rep6.stranded_pos.bed
OL43_20221003.hg38.raw.Output.rep1.stranded_pos.bed
OL43_20221003.hg38.raw.Output.rep2.stranded_pos.bed
OL43_20221003.hg38.raw.Output.rep3.stranded_pos.bed
OL43_20221003.hg38.raw.Output.r

In [4]:
ASSAY=Tewhey_K562_TileMPRA
PREFIX=OL
FP_INPS=($(ls ${FD_INP}/${PREFIX}*hg38*raw*rep*bed))
echo Total: ${#FP_INPS[@]}

Total: 27


**RUN**

In [6]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/coverage_ccres_v4_tmpra.%a.txt \
    --array 0-26 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
ASSAY=Tewhey_K562_TileMPRA
PREFIX=OL

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output directory
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
FD_OUT=${FD_RES}/results/${ASSAY}/coverage_ccres_v4

FP_INPS=($(ls ${FD_INP}/${PREFIX}*hg38*raw*rep*bed))
FP_INP=${FP_INPS[${SLURM_ARRAY_TASK_ID}]}
FN_INP=$(basename ${FP_INP})

FN_OUT=${FN_INP}.gz
FP_OUT=${FD_OUT}/${FN_OUT}

FD_REG=${FD_RES}/results/region/annotation_ccres
FN_REG=ccres_v4.bed.gz
FP_REG=${FD_REG}/${FN_REG}

### Show I/O file
echo "Input  file: " ${FP_INP}
echo
echo "Show the first few lines of the input file"
fun_head ${FP_INP} | head -n 5 
echo

echo "Input  file: " ${FP_REG}
echo
echo "Show the first few lines of the input file"
fun_head ${FP_REG} | head -n 3 
echo

### execute
bedtools map \
    -a ${FP_REG} \
    -b ${FP_INP} \
    -o sum \
    -f 0.5 \
    -F 0.5 \
    -e \
| gzip -c \
> ${FP_OUT}

### Show I/O file
echo
echo "Output file: " ${FP_OUT}
echo
echo "Show the first few lines of the output file"
fun_head ${FP_OUT}
echo

### print end message
timer_stop=`date +%s`
runtime=$(echo "${timer_stop} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 30635243


**Check results**

In [7]:
cat ${FD_LOG}/coverage_ccres_v4_tmpra.0.txt

Hostname:           x2-06-2.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         01-24-24+09:49:07

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/Tewhey_K562_TileMPRA/fragment_count/OL13_20220512.hg38.raw.Input.rep1.stranded_pos.bed

Show the first few lines of the input file
chr11	61787329	61787528	chr11:61554801-61555000	971	+
chr11	61787334	61787533	chr11:61554806-61555005	1267	+
chr11	61787339	61787538	chr11:61554811-61555010	1183	+
chr11	61787344	61787543	chr11:61554816-61555015	1020	+
chr11	61787349	61787548	chr11:61554821-61555020	1138	+

Input  file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_ccres/ccres_v4.bed.gz

Show the first few lines of the input file
chr1	10033	10250	EH38E2776516	0	.	10033	10250	225,225,225	Low-DNase	All-data/Full-classification
chr1	10385	10713	EH38E2776517	0	.	10385	10713	225,225,225	Low-DNase	All-data/Full-classification
chr1	16097	16381	EH38E3951272	0	.	16097	16381	225,225,225	Low-DNa